In [8]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import tabula
import PyPDF2 as pdf

In [15]:
def master_doc(filename):
    frames = []
    length = pdf_page_count(filename)
    new_length = length + 1
    for x in range(3, new_length):
        df = tabula.read_pdf(filename, pandas_options={'header': 2}, pages = x, multi_tables=True, guess = False, stream=True,lattice=False)
        frames.append(df)
    full_df = pd.concat(frames)
    full_df = full_df.reset_index(drop = True)
    full_df['CUSIP NO'] = full_df['CUSIP NO'].str.replace(" ", "")
    full_df['ISSUER NAME'] = full_df["ISSUER NAME"].str.replace('*', "")
    full_df = full_df.drop(full_df.tail(1).index)
    full_df['ISSUER DESCRIPTION'] = full_df['ISSUER DESCRIPTION'].combine_first(full_df['ISSUER DESCRIPTION STATUS'])
    full_df = full_df[['CUSIP NO', 'ISSUER DESCRIPTION', 'ISSUER NAME', 'STATUS']]
    return full_df

In [16]:
def pdf_page_count(filename):
    pdfFileObj = open(filename, 'rb')
    pdfReader = pdf.PdfFileReader(pdfFileObj)
    length = pdfReader.numPages
    return length

In [11]:
df = master_doc('13flist2019q3.pdf')

of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.
 [ipykernel_launcher.py:8]


In [14]:
df.tail(20)

,CUSIP NO,ISSUER DESCRIPTION,ISSUER NAME,STATUS
17814,98980G952,PUT,ZSCALER INC,NaN
17815,98980L101,CL A,ZOOM VIDEO COMMUNICATIONS IN,NaN
17816,98980L901,NaN,ZOOM VIDEO COMMUNICATIONS IN,NaN
17817,98980L951,NaN,ZOOM VIDEO COMMUNICATIONS IN,NaN
17818,989817101,NaN,ZUMIEZ INC,NaN
17819,989817901,NaN,ZUMIEZ INC,NaN
17820,989817951,NaN,ZUMIEZ INC,NaN
17821,98983V106,NaN,ZUORA INC,NaN
17822,98983V906,NaN,ZUORA INC,NaN
17823,98983V956,NaN,ZUORA INC,NaN


In [17]:
def equity_df(master_df):
    equity_df = master_df.loc[master_df['ISSUER DESCRIPTION'].str.contains('COM|SHS')]
    return equity_df